In [61]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configuración de opciones para Chrome
chrome_options = Options()

# Configuración del servicio de ChromeDriver
service = Service(ChromeDriverManager().install())

# Inicialización del WebDriver con el servicio y las opciones configuradas
driver = webdriver.Chrome(service=service, options=chrome_options)

url = "https://www.mercadolibre.com.ar/ofertas#nav-header"

# Abrir la URL
driver.get(url)

# Aceptar cookies si el banner está visible
try:
    cookie_banner = driver.find_element(By.CLASS_NAME, "cookie-consent-banner-opt-out__action")
    if cookie_banner.is_displayed():
        cookie_banner.click()
except:
    pass

# Lista para acumular los datos
data = []

# Procesos a ejecutar con el driver abierto
for i in range(20):
    # Extraer el contenido de la página con BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Encontrar los elementos deseados
    nombres = soup.findAll(class_="promotion-item__title")
    precios = soup.findAll(class_="andes-money-amount__fraction")
    descuentos = soup.findAll(class_="promotion-item__discount-text")
    
    # Acumular los resultados en la lista
    for nombre, precio, descuento in zip(nombres, precios, descuentos):
        data.append({
            'Nombre': nombre.text.strip(),
            'Precio': precio.text.strip(),
            'Descuento': descuento.text.strip()
        })
    
    # Verificar si el botón de siguiente página está disponible
    try:
        boton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '.andes-pagination__button.andes-pagination__button--next'))
        )
        boton.click()
    except:
        print("No hay más páginas para mostrar o el botón no es clickeable.")
        break

# Cerrar el navegador
driver.quit()

# Crear un DataFrame de pandas con los datos recopilados
df = pd.DataFrame(data)

# Guardar el DataFrame en un archivo CSV
df.to_csv('mercadolibre_ofertas.csv', index=False, encoding='utf-8')

print("Datos guardados en 'mercadolibre_ofertas.csv'")


No hay más páginas para mostrar o el botón no es clickeable.
Datos guardados en 'mercadolibre_ofertas.csv'
